In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:

from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)

In [14]:
import asyncio
import random
from typing import Any
from pydantic import BaseModel
from agents import Agent, AgentHooks, RunContextWrapper, Runner, Tool, function_tool


class CustomAgentHooks(AgentHooks):
    def __init__(self, display_name: str):
        self.event_counter = 0
        self.display_name = display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started"
        )

    async def on_end(
        self, context: RunContextWrapper, agent: Agent, output: Any
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} ended with output {output}"
        )

    async def on_handoff(
        self, context: RunContextWrapper, agent: Agent, source: Agent
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

    async def on_tool_start(
        self, context: RunContextWrapper, agent: Agent, tool: Tool
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started tool {tool.name}"
        )

    async def on_tool_end(
        self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} ended tool {tool.name} with result {result}"
        )


###


@function_tool
def random_number(max: int) -> int:
    """
    Generate a random number up to the provided maximum.
    """
    return random.randint(0, max)


@function_tool
def multiply_by_two(x: int) -> int:
    """Simple multiplication by two."""
    return x * 2


class FinalResult(BaseModel):
    number: int


multiply_agent = Agent(
    name="Multiply Agent",
    instructions="Multiply the number by 2 and then return the final result.",
    tools=[multiply_by_two],
    output_type=FinalResult,
    hooks=CustomAgentHooks(display_name="Multiply Agent"),
    model=model,
)
enable_verbose_stdout_logging()
start_agent = Agent(
    name="Start Agent",
    instructions="Generate a random number by calling the random_number.  if generated number is even handoff multiple agent",
    tools=[random_number],  
    hooks=CustomAgentHooks(display_name="Start Agent"),
    model=model,
)


async def main() -> None:
    user_input = input("Enter a max number: ")
    await Runner.run(
        start_agent,
        input=f"Generate a random number between 1 and {user_input}.",
    )

    print("Done!")


if __name__ == "__main__":
    asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000001E3D1E25950>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000001E3D1E25950>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000001E3D1E25950>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object